# Dataset Index Generation
Generates indices for train, validation and test sets

In [1]:
import h5py
# from progressbar import *
import re
import numpy as np

## Options

In [31]:
n_test_files = 400
n_val_files = 100
labels = [0, 1]

## Load dataset

In [24]:
data_path = "/project/rpp-blairt2k/machine_learning/data/HKHybrid/numpy/HKHybrid_e-gamma_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_6Mevts_w_mPMT.hdf5"
f = h5py.File(data_path, "r")

In [25]:
event_labels = np.array(f['labels'])
root_files = np.array(f['root_files']).astype(str)

In [32]:
print(len(event_labels))

6000000


## Find the files of each label and indices of each file

In [6]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [33]:
files_in_labels = {l: sorted(set(root_files[event_labels==l]), key=natural_keys) for l in labels}
idxs_in_files = {f: range(i, i+c) for f,i,c in zip(*np.unique(root_files, return_index=True, return_counts=True))}

In [34]:
for l, f in files_in_labels.items():
    print("label", l,"has", len(f),"files and ", sum([len(idxs_in_files[i]) for i in f]), "indices")

label 0 has 1000 files and  3000000 indices
label 1 has 1000 files and  3000000 indices


## Create the splits

In [35]:
split_files = {"test_idxs":  [f for l in labels for f in files_in_labels[l][:n_test_files]],
               "val_idxs":   [f for l in labels for f in files_in_labels[l][n_test_files:n_test_files+n_val_files]],
               "train_idxs": [f for l in labels for f in files_in_labels[l][n_test_files+n_val_files:]]}
split_idxs = {k: [i for f in v for i in idxs_in_files[f]] for k, v in split_files.items()}

In [36]:
for s in split_files.keys():
    print(s,"has", len(split_files[s]),"files and", len(split_idxs[s]),"indices")

test_idxs has 800 files and 2400000 indices
val_idxs has 200 files and 600000 indices
train_idxs has 1000 files and 3000000 indices


In [37]:
# Verify that all events are uniquely accounted for
all_indices = np.concatenate(list(split_idxs.values()))
print(len(event_labels))
print(len(all_indices))
print(len(set(all_indices)))

6000000
6000000
6000000


['/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/gamma/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_gamma_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_0.root',
 '/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/gamma/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_gamma_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_1.root',
 '/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/gamma/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_gamma_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_2.root',
 '/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/gamma/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_gamma_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_3.root',
 '/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/gamma/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_gamma_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_4.root',
 '/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/gamma/E0t

## Save file

In [39]:
np.savez('/project/rpp-blairt2k/machine_learning/data/HKHybrid/numpy/HKHybrid_e-gamma_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_idxs_w_mPMT.npz', **split_idxs)